<a href="https://colab.research.google.com/github/jiruneko/IntuitionLLM/blob/main/IntuitionLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "microsoft/Phi-3-mini-4k-instruct"

# 利用可能なデバイス判定
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenizer 読み込み
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Model 読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # GPU があれば自動で割り当て / CPUのみなら None
    device_map="auto" if device == "cuda" else None,
    # GPU 環境なら軽量 dtype / CPU なら float32 のまま
    torch_dtype=(
        torch.bfloat16
        if device == "cuda" and torch.cuda.is_bf16_supported()
        else (torch.float16 if device == "cuda" else torch.float32)
    ),
    trust_remote_code=False,  # 必要なら True に変更可
)

# 推論用パイプライン（任意だけど便利）
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto" if device == "cuda" else None,
)

# 動作確認
out = pipe(
    "Hello, Phi-3!",
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
)
print(out[0]["generated_text"])

In [ ]:
from transformers import pipeline

# パイプラインの作成
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

In [ ]:
# プロンプトを設定
messages = [
    # 鶏に関するおもしろいジョークを作ってください。
    {"role": "user", "content": "鶏に関するおもしろいジョークを作ってください。"}
]

# 出力を生成
output = generator(messages)
print(output[0]["generated_text"])

In [ ]:
prompt = "悲惨なガーデニングでの災難について、サラに謝罪のメールを書いてください。どうしてそんなことが起きたのか説明してください。"

# 入力プロンプトをトークン化する
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# テキストの生成
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=20
)

# 出力の表示
print(tokenizer.decode(generation_output[0]))

In [ ]:
print(input_ids)

In [ ]:
for id in input_ids[0]:
  print(tokenizer.decode(id))

In [ ]:
generation_output

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

In [ ]:
colors_list = [
    "102;194;165", "252;141;98", "141;160;203",
    "231;138;195", "166;216;84", "255;217;47"
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f"\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m" +
            tokenizer.decode(t) +
            "\x1b[0m",
            end=" "
        )

In [ ]:
from transformers import AutoModel, AutoTokenizer

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-xsmall")

# 言語モデルの読み込み
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# 文をトークン化
tokens = tokenizer("Hello world", return_tensors="pt")

# トークンを処理
output = model(**tokens)[0]

In [ ]:
output.shape

In [ ]:
for token in tokens["input_ids"][0]:
  print(tokenizer.decode(token))

In [ ]:
output

In [ ]:
from sentence_transformers import SentenceTransformer

# モデルの読み込み
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# テキストをテキスト埋め込みに変換
vector = model.encode("Best movie ever!")

In [ ]:
vector.shape

In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")

In [ ]:
model.most_similar([model["king"]], topn=11)